In [60]:
import pandas as pd
import os
import re
import subprocess
import os
import re
import pandas as pd
from multiprocessing import Pool, cpu_count

import warnings
warnings.filterwarnings("ignore")

In [61]:
# in the below list mention the AESes that you want to balcklist from cm search Fasta
input_cm = "S19"
blacklisted_aeses = ["S19"]

## Execute Below cells for CMSearch result on the above inputs

In [66]:
# concat all fasta
fasta_path = "/home/sumon/workspace/git_repos/aes_db/data/all_fasta"
FASTA_PATH = "/home/sumon/workspace/git_repos/aes_db/tmp/bac_ribosome.fasta"
black_list_string = "|".join(blacklisted_aeses)


with open(FASTA_PATH, 'w') as dst_file:
    for fasta_file in os.listdir(fasta_path):
        if re.search(black_list_string, fasta_file):
            # print(f"Skipping: {fasta_file}")
            continue
        
        fasta_full_path = os.path.join(fasta_path, fasta_file)
        with open(fasta_full_path, 'r') as fasta_file:
            for line in fasta_file.readlines():
                dst_file.writelines(line)
        
        dst_file.writelines("\n\n")


In [76]:

def cm_search(pair, fasta_path=FASTA_PATH):
    (cm_path, out_file) = pair
    command = ["/home/R2DT1_4/infernal/infernal-1.1.5/src/cmsearch", "--tblout", out_file, cm_path, fasta_path]
    result = subprocess.run(command, 
                            capture_output=True, text=True)
    with open(out_file.replace(".tblout", ".out"), 'w') as f:
        f.writelines(result.stdout)
    return out_file, result.stdout

In [77]:
cm_dir = "/home/sumon/workspace/git_repos/aes_db/data/all_cms"
search_out = "/home/sumon/workspace/git_repos/aes_db/tmp"
list_of_cms = [(os.path.join(cm_dir, i), os.path.join(search_out, i + ".tblout")) for i in os.listdir(cm_dir) if re.match(input_cm, i)]

ls = []
for pair in list_of_cms:
    outfile, stdout = cm_search(pair)
    # print(f"cmsearch out: {outfile}")
    ls.append([outfile, stdout])

## See result

In [78]:
for path, out in ls:
    print(f"File Generated: {path}\n")
    print(out)

File Generated: /home/sumon/workspace/git_repos/aes_db/tmp/S19_0.cm.tblout

# cmsearch :: search CM(s) against a sequence database
# INFERNAL 1.1.5 (Sep 2023)
# Copyright (C) 2023 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query CM file:                         /home/sumon/workspace/git_repos/aes_db/data/all_cms/S19_0.cm
# target sequence database:              /home/sumon/workspace/git_repos/aes_db/tmp/bac_ribosome.fasta
# tabular output of hits:                /home/sumon/workspace/git_repos/aes_db/tmp/S19_0.cm.tblout
# number of worker threads:              4
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       aes_19_0  [CLEN=35]
Hit scores:
 rank     E-value  score  bias  sequence                                                                             start    end   mdl trunc   gc  description
 ----   --------- ------ ----

## For parsing cmsearch global result 

In [66]:

def parse_to_list(st):
    return list(filter(None, st.split(" ")))

def parse_file_to_df(file_path):
    headers = ['target_name', "target_accession", "query_name", "query_accession", "mdl", "mdl_from", "mdl_to", "seq_from", "seq_to", "strand", "trunc", "pass", "gc", "bias", "score", "e_value", "inc", "des"]
    data = []
    with open(file_path) as fp:
        for line in fp.readlines():
            if line.startswith("#"):
                continue
            data.append(parse_to_list(line.strip()))
    df = pd.DataFrame(data, columns=headers)
    current_cm = os.path.basename(file_path)
    # df.loc[:, "filename"] = current_cm
    return df

In [68]:
os.path.join(base_path, tblout)

'/home/sumon/workspace/git_repos/aes_db/data/all_tsv/P7_0.cm.tblout'

In [67]:
parse_file_to_df(os.path.join(base_path, tblout))

,target_name,target_accession,query_name,query_accession,mdl,mdl_from,mdl_to,seq_from,seq_to,strand,trunc,pass,gc,bias,score,e_value,inc,des


In [73]:
all_data = []
base_path = "/home/sumon/workspace/git_repos/aes_db/data/all_tsv"
for tblout in os.listdir(base_path):
    if not tblout.endswith(".tblout"):
        continue
    df = parse_file_to_df(os.path.join(base_path, tblout))
    
    if df.empty:
        continue
    df.loc[:, "filename"] = tblout
    df.loc[:, "hit"] = df.target_name.str.split("|", expand=True)[1]
    all_data.append(df)

In [83]:
master_df = pd.concat(all_data)

In [88]:
master_df.loc[:, "query_permutation"]  = master_df.filename.str.split(".", expand=True)[0]
master_df.loc[:, "query_aes"]  = master_df.query_permutation.str.split("_", expand=True)[0]

master_df.loc[:, "target_permutation"]  = master_df.target_name.str.split("|", expand=True)[1]
master_df.loc[:, "target_aes"]  = master_df.target_permutation.str.split("_", expand=True)[0]

In [92]:
no_self_hit = master_df[master_df.target_aes != master_df.query_aes].sort_values("e_value")

In [94]:
no_self_hit.to_csv("tmp/cmsearch_self_hit_removed.csv", index=False)